In [155]:
# Importing relevant librairies
import numpy as np
import pandas as pd
import yfinance as yf
import os
pd.set_option('display.max_columns', None)

In [197]:
def load_csv(ticker: str):
    """
    Creating a function that would allow us to load a csv containing all relevant features of a given ticker
    ticker should be in capital
    Start and end should be in the be following format: %YYYY-%MM-%DD
    """

    #Loading all relevant csv files

    #Loading stock return
    ticker_df = pd.read_csv(f'../raw_data/stocks_return/adj_return_{ticker}.csv')
    ticker_df.rename(columns={'timestamp':'date'}, inplace=True)
    ticker_df.sort_values('date', ascending=True, inplace=True)
    ticker_df.reset_index(inplace=True, drop=True)

    if '1999-12-31' in ticker_df['date'].values:
        ticker_df = ticker_df[ticker_df['date']>'1999-12-31']
        ticker_df.reset_index(inplace=True, drop=True)

    #Loading stocks eps
    ticker_eps = pd.read_csv(f'../raw_data/eps/data_{ticker}.csv', index_col=0)
    ticker_eps['date'] = ticker_eps['reportedDate'].copy()
    ticker_eps.sort_values('date', inplace=True)
    ticker_eps['reportedDate']= pd.to_datetime(ticker_eps['reportedDate'])
    ticker_eps['year'] = pd.DatetimeIndex(ticker_eps['reportedDate']).year

    if 2000 in ticker_eps['year'].values:
        ticker_eps = ticker_eps[ticker_eps['year']>=2000]
        ticker_eps.reset_index(inplace=True, drop=True)

    ticker_eps.drop(columns=['year','fiscalDateEnding', 'estimatedEPS', 'surprise', 'reportedDate'], inplace=True)

    #Loading all macros and commodities
    gold = pd.read_csv('../raw_data/macro/gold.csv', index_col=0)
    us_dollar = pd.read_csv('../raw_data/macro/usd.csv', index_col=0)

    credit_spread = pd.read_csv('../raw_data/macro/us_yields.csv', index_col=0)
    oil = pd.read_csv('../raw_data/macro/oil.csv', index_col=0)
    orders = pd.read_csv('../raw_data/macro/orders.csv', index_col=0)
    ffunds_rate = pd.read_csv('../raw_data/macro/fed_funds.csv', index_col=0)
    unemployment = pd.read_csv('../raw_data/macro/unemployment.csv', index_col=0)
    inflation_expectation = pd.read_csv('../raw_data/macro/inflation_expectation.csv', index_col=0)
    non_farm_payroll = pd.read_csv('../raw_data/macro/non_farm_payroll.csv', index_col=0)
    cpi = pd.read_csv('../raw_data/macro/cpi.csv', index_col=0)
    retails = pd.read_csv('../raw_data/macro/retail_sales.csv', index_col=0)
    gdp_capita = pd.read_csv('../raw_data/macro/gdp_per_capita.csv', index_col=0)


    #Creating our final df and merging with relevant files
    final_df = ticker_df.copy()
    final_df = final_df.merge(ticker_eps, how='outer', on='date')
    final_df = final_df.merge(credit_spread, how='outer')
    final_df = final_df.merge(oil, how='outer', on='date')
    final_df = final_df.merge(orders, how='outer')
    final_df = final_df.merge(ffunds_rate, how='outer')
    final_df = final_df.merge(unemployment, how='outer')
    final_df = final_df.merge(inflation_expectation, how='outer')
    final_df = final_df.merge(non_farm_payroll, how='outer')
    final_df = final_df.merge(cpi, how='outer')
    final_df = final_df.merge(retails, how='outer')
    final_df = final_df.merge(gdp_capita, how='outer')
    final_df = final_df.merge(gold, how='outer', on='date')
    final_df = final_df.merge(us_dollar, how='outer', on='date')

    #Sorting by chronological order and resetting index
    final_df = final_df.sort_values('date', ascending=True)
    final_df.reset_index(drop=True, inplace=True)

    #Since not all features have daily data, we forward filled the missing values
    final_df['orders'] = final_df['orders'].fillna(method='ffill')
    final_df['retail_sales'] = final_df['retail_sales'].fillna(method='ffill')
    final_df['gdp_per_capita'] = final_df['gdp_per_capita'].fillna(method='ffill')
    final_df['CPI'] = final_df['CPI'].fillna(method='ffill')
    final_df['non_farm_payroll'] = final_df['non_farm_payroll'].fillna(method='ffill')
    final_df['inf_exp'] = final_df['inf_exp'].fillna(method='ffill')
    final_df['unemployment_rate'] = final_df['unemployment_rate'].fillna(method='ffill')
    final_df['reportedEPS'] = final_df['reportedEPS'].fillna(method='ffill')
    final_df['surprisePercentage'] = final_df['surprisePercentage'].replace(['None'], '0')
    final_df['surprisePercentage'] = final_df['surprisePercentage'].replace([np.nan], '0')
    
    #Dropping null values
    final_df.dropna(inplace=True)
    final_df.reset_index(drop=True, inplace=True)

    return final_df

In [198]:
# reportedEPS etaler sur toutes les dates
# surprise percentage mettre a 0
# dans le pipe il faut pas toucher au targert ?

In [199]:
apple = load_csv('AAPL')
apple.isnull().sum()

date                  0
open                  0
high                  0
low                   0
close                 0
adjusted_close        0
volume                0
dividend_amount       0
split_coefficient     0
reportedEPS           0
surprisePercentage    0
10Y_yield             0
2Y_yield              0
10_2_spread           0
oil_price             0
orders                0
fed_funds             0
unemployment_rate     0
inf_exp               0
non_farm_payroll      0
CPI                   0
retail_sales          0
gdp_per_capita        0
gold_price            0
usd_price             0
dtype: int64

In [196]:
apple['surprisePercentage'] = apple['surprisePercentage'].replace(['None'], '0')

In [176]:
apple['surprisePercentage'] = apple['surprisePercentage'].replace([np.nan], '0')

In [181]:
apple['surprisePercentage'].unique()

array(['0', '33.3333', '100', '66.6667', '25', '16.6667', '28.5714',
       '45.4545', '30', '8.6957', '13.3333', '12.5', '18.75', '11.7647',
       '73.3333', '37.1429', '13.6364', '13.7931', '19.4805', '18.1818',
       '33.7349', '-4.717', '36.5517', '23.0769', '-10.1351', '-0.8',
       '2.6042', '0.6993', '1.9048', '3.5088', '2.9851', '13.6986',
       '4.065', '8.3969', '17.6923', '7.8704', '2.2099', '4.2553',
       '1.548', '-5', '2.8986', '0.6024', '4.6729', '3.9604', '6.3694',
       '10.6952', '0.7772', '1.487', '7.3394', '4.2857', '0.9615',
       '5.0847', '3.7736', '7.0423', '9.6491', '14.2857', '27.451',
       '19.1489', '41.4141', '28.7129', '11.1111', '6.2937', '3.4483',
       '1.5748'], dtype=object)